In [21]:
import os
import json
from collections import defaultdict, Counter

In [22]:
file_path = '../ensemble/n_best'

json_file_path = []
json_files = []

total_count = 0
for json_path in os.listdir(file_path) :
    if json_path.endswith('.json') :
        total_count = total_count + 1
        json_file_path.append(json_path)

for file in json_file_path :
    fp = os.path.join(file_path,file)
    with open(fp , "r" , encoding='utf-8') as json_file :
        json_data = json.load(json_file)
        json_files.append(json_data)

print(json_file_path)
print(json_file)

['nbest_predictions.json']
<_io.TextIOWrapper name='../ensemble/n_best/nbest_predictions.json' mode='r' encoding='utf-8'>


In [23]:
key_list=list(json_files[0].keys())                   # predictions.json의 key값들 설정
bin=defaultdict(list)                                 # 확률값들을 담을 dict생성-> value : list
answer_dict=defaultdict(list)                         # 최종 답변을 넣을 dict 생성 -> value : list

for i in range(len(key_list)):                        #defaultdict : key만 선언하면 value는 '' 빈 값
    bin[key_list[i]] = []                                 
    answer_dict[key_list[i]] = []

for json_file in json_files:                          # N개의 nbest_predictions 파일들
    for kl in key_list:                               # 600개의 key
        for i in range(total_count):                            # nbest_size : 20                       
            bin[kl].append((json_file[kl][i]['text'],json_file[kl][i]['probability']))                       

In [24]:
for kl in key_list:                                   # 600개의 키
    answer_hubo={}
    for i in range(len(bin[kl])):                  
        if bin[kl][i][0] not in answer_hubo.keys():   # 빈 객체에서 없다면 key 추가
            answer_hubo[bin[kl][i][0]]=bin[kl][i][1]
        else:
            answer_hubo[bin[kl][i][0]]+=bin[kl][i][1] # 있다면 확률값 더하기 -> 이렇게 하지 않으면 dictionary는 자동으로 중복제거(set)
    max_answer=max(answer_hubo.keys(),key= lambda prob : answer_hubo[prob])     #최대 확률값을 답변으로
    answer_dict[kl]=max_answer

In [25]:
new_nbest_json_path = "../ensemble/output/nbest_ensemble.json"           # nbest_ensemble.json 저장

with open(new_nbest_json_path, 'w', encoding='utf-8') as file:
    json.dump(answer_dict, file, indent=4,ensure_ascii=False)